In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np
import tensorflow as tf
from datasets import Dataset
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score

In [2]:
# Set the path to the file you'd like to load
file_path = "df_total.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "kevinmorgado/spanish-news-classification",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

df.head()

,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra


In [3]:
# --- PREPARACIÓN DEL DATASET PARA BERT ---
from sklearn.preprocessing import LabelEncoder

print("--- 1. Preprocesamiento: Renombrar y Mapear Etiquetas ---")

# 1. Renombrar columnas: 'news' (contenido) -> 'text', 'Type' (clasificación) -> 'label'
df.rename(columns={'news': 'text', 'Type': 'label'}, inplace=True)

# Limpiar valores nulos y resetear el índice
df = df[['text', 'label']].dropna().reset_index(drop=True)

# 2. Mapeo de categorías a enteros (Necesario para el modelo de clasificación)
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

categories = le.classes_.tolist()
NUM_LABELS = len(categories)

# Crear el mapeo de categorías para referencia futura (opcional)
label_map = dict(zip(categories, range(NUM_LABELS)))

print(f"Dataset final con {len(df)} muestras.")
print(f"Clases detectadas: {categories} ({NUM_LABELS} clases)")
print(f"Mapeo (Etiqueta Original -> Entero): {label_map}")

--- 1. Preprocesamiento: Renombrar y Mapear Etiquetas ---
Dataset final con 1217 muestras.
Clases detectadas: ['Alianzas', 'Innovacion', 'Macroeconomia', 'Otra', 'Regulaciones', 'Reputacion', 'Sostenibilidad'] (7 clases)
Mapeo (Etiqueta Original -> Entero): {'Alianzas': 0, 'Innovacion': 1, 'Macroeconomia': 2, 'Otra': 3, 'Regulaciones': 4, 'Reputacion': 5, 'Sostenibilidad': 6}


In [5]:
# --- 2. Conversión y División de Conjuntos ---

print("--- División de Conjuntos (Train, Validation, Test) ---")

# Conversión a formato Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Definir las proporciones
test_size_ratio = 0.2  # 20% para Test+Validation
valid_size_ratio = 0.5 # 50% de ese 20% (es decir, 10% del total)

hf_dataset_splits = hf_dataset.train_test_split(test_size=test_size_ratio, seed=42)
# Dividir el conjunto de prueba/validación
test_valid_split = hf_dataset_splits['test'].train_test_split(test_size=valid_size_ratio, seed=42)

train_dataset = hf_dataset_splits['train']
validation_dataset = test_valid_split['train']
test_dataset = test_valid_split['test']

print(f"Tamaño de Entrenamiento: {len(train_dataset)}")
print(f"Tamaño de Validación: {len(validation_dataset)}")
print(f"Tamaño de Prueba: {len(test_dataset)}")

--- División de Conjuntos (Train, Validation, Test) ---
Tamaño de Entrenamiento: 973
Tamaño de Validación: 122
Tamaño de Prueba: 122


In [6]:
# --- 3. Tokenización (Paso 2 del desarrollo) ---

from transformers import AutoTokenizer, logging

# Suprimir advertencias de Hugging Face para un entorno más limpio
logging.set_verbosity_error()

print("--- Tokenización de Datos para BERT ---")

# Definiciones de configuración
MODEL_NAME = 'dccuchile/bert-base-spanish-wwm-cased' # BERT pre-entrenado en español
MAX_LENGTH = 128
BATCH_SIZE = 16
# NUM_LABELS ya está definida en una celda anterior

# Cargar Tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Función de Tokenización:
def tokenize_function(examples):
    # Truncation y padding son cruciales para uniformar la entrada de BERT
    return tokenizer(examples["text"], 
                     truncation=True, 
                     padding='max_length', 
                     max_length=MAX_LENGTH)

# Aplicar tokenización a los conjuntos
# CORRECCIÓN: Quitamos '__index_level_0__' de remove_columns, ya que no existe.
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
tokenized_validation = validation_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
tokenized_test = test_dataset.map(tokenize_function, batched=True, remove_columns=['text'])

print("Tokenización completa. Los datos están listos para ser convertidos a tensores de TensorFlow.")

--- Tokenización de Datos para BERT ---


Map:   0%|          | 0/973 [00:00<?, ? examples/s]

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

Tokenización completa. Los datos están listos para ser convertidos a tensores de TensorFlow.


In [7]:
# --- 4. Carga del Modelo, Conversión a TensorFlow Dataset y Compilación ---

from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf 

# RE-DEFINICIÓN DE VARIABLES CLAVE (Asegura que estén disponibles)
MODEL_NAME = 'dccuchile/bert-base-spanish-wwm-cased'
BATCH_SIZE = 16
LEARNING_RATE = 2e-5 # Tasa de aprendizaje óptima para fine-tuning
EPOCHS = 3 
# Nota: NUM_LABELS debe haber sido definido en la Celda 3

print("--- Carga del Modelo Pre-entrenado (Transfer Learning) y Compilación ---")

# Cargar el modelo BERT para la tarea de Clasificación de Secuencias
model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=NUM_LABELS
)

# Convertir a tensores de TensorFlow (tf.data.Dataset)
# Este paso es crucial para optimizar el entrenamiento en Keras/TensorFlow
tf_train = tokenized_train.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['label'],
    shuffle=True,
    batch_size=BATCH_SIZE
)
tf_validation = tokenized_validation.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['label'],
    shuffle=False,
    batch_size=BATCH_SIZE
)

# Compilar el modelo (Ajuste de hiperparámetros - Paso 3 del desarrollo)
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

print("Modelo BERT compilado. ¡Listo para el Fine-Tuning!")

--- Carga del Modelo Pre-entrenado (Transfer Learning) y Compilación ---


/Users/jtellez/Library/Caches/pypoetry/virtualenvs/tellez-guillent-practica-i-WaD7jEWf-py3.13/lib/python3.13/site-packages/datasets/arrow_dataset.py:405: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


Modelo BERT compilado. ¡Listo para el Fine-Tuning!


In [ ]:
# --- 5. Entrenamiento (Fine-Tuning) ---

print(f"--- Iniciando Fine-Tuning de BERT por {EPOCHS} épocas ---")

# Entrenar el modelo
history = model.fit(
    tf_train,
    validation_data=tf_validation,
    epochs=EPOCHS
)

print("\n✅ Entrenamiento (Fine-Tuning) completado.")

--- Iniciando Fine-Tuning de BERT por 3 épocas ---
Epoch 1/3


In [ ]:
# --- 6. Evaluación del Rendimiento (Paso 4 del desarrollo) ---

from sklearn.metrics import accuracy_score, recall_score, f1_score

print("--- Evaluación Final en Conjunto de Prueba ---")

# Preparar el conjunto de prueba en formato tf.data.Dataset
tf_test = tokenized_test.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['label'],
    shuffle=False,
    batch_size=BATCH_SIZE
)

# Generar predicciones (logits)
print("Generando predicciones en el conjunto de prueba...")
logits = model.predict(tf_test).logits
predictions = tf.argmax(logits, axis=-1).numpy()
true_labels = np.array(test_dataset['label'])

# Calcular métricas requeridas
accuracy = accuracy_score(true_labels, predictions)
# Usamos 'weighted' para el promedio de métricas en clasificación multi-clase
recall = recall_score(true_labels, predictions, average='weighted') 
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"\n--- Resultados Finales de Evaluación ---")
print(f"Precisión (Accuracy): {accuracy:.4f}")
print(f"Recall (Sensibilidad): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")